In [49]:
import numpy as np
import pandas as pd
from pickle import TRUE
from random import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

PRED_ERROR = 0.1

class Neural_Network:

    # initialize the neural network
    def __init__(self, num_inputs, num_hidden , num_outputs):
        self.num_inputs = num_inputs   # num_inputs = number of nodes in the input layer
        self.num_hidden = num_hidden   # num_hidden = number of nodes in the hidden layers i.e: hidden_layers_size [20, 30] it has 2 hidden layers first hidden layer has
                                       # 20 nodes and the second hidden layer has 30 nodes
        self.num_outputs = num_outputs # num_outputs = number of nodes in the output layer
        layers = [self.num_inputs] + self.num_hidden + [self.num_outputs] # obtain a list that represents the size of each layer in the neural network.
                                       # i.e: self.num_inputs = 4, self.num_hidden = [8, 16], and self.num_outputs = 3, then layers will be equal to [4, 8, 16, 3]

        # initiate weights
        self.weights = []      # an empty weights list
        self.prev_weights = [] # storing previous weights list
        self.momentumterm = []

        # initialize the weights randomly
        for i in range(len(layers) - 1):
            w = np.random.rand(layers[i] , layers[i+1]) # list of weight matrices which has rows and columns to the corresponding layer and the one after it
            self.weights.append(w)

        activations = []             # empty list created to store the activations (outputs) of each layer of the neural network during the forward propagation
        for i in range(len(layers)): # iterating through layers and storing them in the activation list
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations

        derivatives = [] # list to store the derivatives of the weights between each layer during backpropagation
        for i in range(len(layers)-1):
            d = np.zeros((layers[i], layers[i+1]))
            derivatives.append(d)
        self.derivatives = derivatives

    def feedforward(self, inputs):
        activations = inputs # Set the input layer activations to the input values
        self.activations[0]= inputs # Store the input layer activations in self.activations[0]
        for i, w in enumerate(self.weights): # Iterate over the weights while keeping track of the index
            # Calculate net inputs to each layer by taking the dot product of the activations and weights for that layer
            net_inputs = np.dot(activations, w)
            # Calculate the activation of the layer using the sigmoid activation function
            activations = self.sigmoid(net_inputs)
            # Store the activations for each layer in self.activations
            self.activations[i+1] = activations
        return activations

    def backpropagation(self, error, verbose = False):
        # Loop over the layers in reverse order
        for i in reversed(range(len(self.derivatives))):
            # Get the activations for the current layer
            activations = self.activations[i+1]
            # Calculate delta, the error multiplied by the derivative of the activation function
            delta = error * self.sigmoid_derivative(activations)
            # Reshape delta to have dimensions (n, 1) for later dot product operation
            delta_reshaped = delta.reshape(delta.shape[0], -1).T
            # Get the activations for the previous layer
            current_activations = self.activations[i]
            # Reshape activations to have dimensions (n, 1) for later dot product operation
            current_activations_reshaped = current_activations.reshape(current_activations.shape[0], -1)
            # Calculate the derivative for the current layer
            self.derivatives[i] = np.dot(current_activations_reshaped, delta_reshaped)
            # Calculate the error for the previous layer using the weights and delta for the current layer
            error = np.dot(delta, self.weights[i].T)

            #verbose is a boolean parameter that is used to control whether or not to print out intermediate results during the backpropagation process
            if verbose:
                print("Derivatives for W{}: {}".format(i, self.derivatives[i]))

            # Return the error for the input layer
        return error

   # Update the weights
    def update_weights(self, learning_rate):
        for i in range(len(self.weights)):
            weights = self.weights[i] # Get the current weights
            derivatives = self.derivatives[i] # Get the derivatives of the current layer
            momentum = self.momentumterm[i]
            weights += momentum + derivatives * learning_rate # Update the weights with the momentum and the derivatives multiplied by the learning rate

    def momentum_fct(self, momentum):
        self.saveprev_weights(self.weights)
        for i in range(len(self.weights)):
            weights = self.weights [i]
            savedprev_weights = self.prev_weights[i]
            x = momentum * ( weights - savedprev_weights )
            self.momentumterm.append(x)

    # Save the current weights of the neural network to the prev_weights
    def saveprev_weights(self, currentweights):
        self.prev_weights.clear()
        for i in range(len(currentweights)):
            self.prev_weights.append(currentweights[i])

    def train(self, inputs, targets, epochs, learning_rate, momentum, verbose = False):

        for i in range(epochs):
            sum_error = 0
            for input, target in zip(inputs, targets):

                output = self.feedforward(input)

                #calculate error
                error = (target - output) # desired - actual output

                #back propagation
                self.backpropagation(error)

                #apply momentum formula to update momentum
                self.momentum_fct(momentum)

                #apply gradient descent
                self.update_weights(learning_rate)

                sum_error += self.mse_fct(target, output)
            #report error
            if verbose:
                print("Error: {} at epoch {} ".format(sum_error / len(inputs), i+1))

    def predict(self, testing_set, testing_output_set):
        predictionTable = []
        predictionTable1 = []
        x = 0
        y = 0
        x1 = 0
        y1 = 0
        for i , target in enumerate(testing_output_set):
            output = self.feedforward(testing_set)
            error = self.mse_fct(target , output[i])

            if error <= PRED_ERROR:
                x += 1
            else:
                y += 1
                if np.round_(output[i]) == 0:
                    x1 += 1
                else:
                    y1 += 1
            predictionTable = [x, y]
            predictionTable1 = [x1, y1]
        return predictionTable , predictionTable1

    def mse_fct(self, target, output):
        return np.average((target-output)**2)

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def sigmoid(self,x):
        return (1 / (1 + np.exp(-x)))

    def normalize(a):
        for i in range(a.shape[0]):
            for j in range(a.shape[1]):
                a[i,j] = (a[i,j] - a.min())/(a.max() - a.min())
        return a

if __name__ == "__main__":

    data = pd.read_csv(r"/content/sample_data/wisc_bc_data.csv")
    # Display the number of rows and columns
    print("Shape of the dataset:", data.shape)
    print(" ")

    # Display data types and non-null counts
    print(data.info())

    # Assuming your DataFrame is named 'data'
    diagnosis_counts = data['diagnosis'].value_counts()

    print(" ")

    print("Number of Benign (B) cases:", diagnosis_counts['B'])
    print("Number of Malignant (M) cases:", diagnosis_counts['M'])

    data = data.drop('id', axis=1)
    data['diagnosis'].replace({'M': 1, 'B': 0}, inplace=True)

    print(" ")
    print("-------------------------------------------------------------------")
    print("After removing the 'id' column, this is the information of the data")
    print("-------------------------------------------------------------------")
    print(" ")

    # Display the number of rows and columns
    print("Shape of the dataset:", data.shape)
    print(" ")

    # Display data types and non-null counts
    print(data.info())

    # Assuming your DataFrame is named 'data'
    diagnosis_counts = data['diagnosis'].value_counts()

    print(" ")
    print("---------------------------------")
    print("We replaced 'B' by 0 and 'M' by 1")
    print("---------------------------------")
    print(" ")

    print("Number of Benign (B) cases:", diagnosis_counts[0])
    print("Number of Malignant (M) cases:", diagnosis_counts[1])

    print(" ")
    print("--------------------------------------------------------")
    print("We divided the dataset into training set and testing set")
    print("--------------------------------------------------------")
    print(" ")

    data = np.array(data)

    TRAINING_PERCENTAGE = 80
    n_epochs = 5

    # Set the number of times the data will be shuffled before each training
    n_datashuffle = 1
    LearningRate = 0.9
    Momentum = 0.6

    Neural_Network.normalize(data)

    data_training =(data[:int(TRAINING_PERCENTAGE/100 * data.shape[0])])
    data_testing =(data[int(TRAINING_PERCENTAGE/100 * data.shape[0]):])

    mlp = Neural_Network(30, [1] , 1)

    # Shuffle the data and train the neural network multiple times
    for i in range(n_datashuffle):

        # Print the shapes of the training and validation sets
        print(data_training.shape)
        print(data_testing.shape)

        print(" ")

        # Extract the inputs and outputs from the training and validation sets
        trainingSet = data_training[:,:30]
        testing_set = data_testing[:,:30]
        trainingTargetSet = data_training[:,30]
        testing_output_set = data_testing[:,30]

        # Train the neural network using the training data
        mlp.train(trainingSet, trainingTargetSet, n_epochs, LearningRate , Momentum, TRUE)

        # Shuffle the training data for the next iteration
        np.random.shuffle(data_training)

    # Predict validation set using the trained model
    predTable, predTable1 = mlp.predict(testing_set, testing_output_set)

    print(" ")

    # Print predicted table and calculate accuracy
    # print(predTable)
    # print(" ")

    accuracyPercentage = predTable[0]*100/(predTable[0] + predTable[1])
    accuracyPercentage="%.4f" % accuracyPercentage
    print("The accuracy is {} %".format (accuracyPercentage))
    print(" ")

    # Print mispredicted values
    print("The number of values mispredicted is: \n (0, 1) \n" , predTable1)

    # # Plots the training and validation accuracy over epochs.
    # plt.plot(accuracyPercentage)
    # plt.title("Model Accuracy")
    # plt.xlabel('Epoch')
    # plt.ylabel('Accuracy')
    # plt.legend(['train', 'validation'], loc='best')
    # plt.show()

    # errors = mlp.train(trainingSet, trainingTargetSet, n_epochs, LearningRate , Momentum, TRUE)
    # plt.plot(errors)
    # plt.xlabel('Epochs')
    # plt.ylabel('Error')
    # plt.title('Training Error Curve')
    # plt.show()

    # plt.scatter(trainingSet[:,0], trainingSet[:,1], c = trainingTargetSet, cmap = 'winter', alpha = 0.5)
    # # trainingSet[:,0] and trainingSet[:,1] are arrays of x and y coordinates for the training data points.
    # # c = trainingTargetSet is an array of color values corresponding to each data point, and it is used to color the points on the plot.
    # plt.scatter(testing_set[:,0], testing_set[:,1], c=testing_output_set, cmap='hot', alpha=0.5)
    # # testing_set[:,0] and testing_set[:,1] are arrays of x and y coordinates for the testing data points.
    # # c = testing_output_set is an array of color values corresponding to each data point, and it is used to color the points on the plot.
    # plt.show() # displays the plot on the screen.

Shape of the dataset: (569, 32)
 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float